In [482]:
import pandas as pd
import numpy as np

In [483]:
delivery=pd.read_csv('delivery_partner.csv')
food=pd.read_csv('food.csv')
menu=pd.read_csv('menu.csv')
order_details=pd.read_csv('order_details.csv')
order=pd.read_csv('orders.csv')
resturant=pd.read_csv('restaurants.csv')
users=pd.read_csv('users.csv')

In [484]:
delivery.columns

Index(['partner_id', 'partner_name'], dtype='object')

In [485]:
food.columns

Index(['f_id', 'f_name', 'type'], dtype='object')

In [486]:
menu.columns

Index(['menu_id', 'r_id', 'f_id', 'price'], dtype='object')

In [487]:
order_details.columns

Index(['id', 'order_id', 'f_id'], dtype='object')

In [488]:
order.columns

Index(['order_id', 'user_id', 'r_id', 'amount', 'date', 'partner_id',
       'delivery_time', 'delivery_rating', 'restaurant_rating'],
      dtype='object')

In [489]:
resturant.columns

Index(['r_id', 'r_name', 'cuisine'], dtype='object')

In [490]:
users.columns

Index(['user_id', 'name', 'email', 'password'], dtype='object')

# Zomato Case Study Problems-

Code 

1. Find customers who have never ordered

2. Average Price/dish

3. Find top restautant in terms of number of orders for a given month

4. restaurants with monthly sales > 4000 

5. Show all orders with order details for a particular customer in a particular date range

6. Find restaurants with max repeated customers

7. Month over month revenue growth of swiggy

8. Customer -> favorite food

# Find customers who have never ordered

In [491]:
ordered_user = order['user_id'].unique()
never_ordered_users = users[~users['user_id'].isin(ordered_user)]
print(f"who have never ordered users is:\n{never_ordered_users}")


who have never ordered users is:
   user_id     name              email password
5        6  Anupama  anupama@gmail.com   46rdw2
6        7  Rishabh  rishabh@gmail.com   4sw123


# Average Price/dish

In [492]:
avg_price = menu['price'].mean()
print(f"Average price is: \n {avg_price}")


Average price is: 
 210.33333333333334


In [493]:
menu.columns

Index(['menu_id', 'r_id', 'f_id', 'price'], dtype='object')

In [494]:
food.columns

Index(['f_id', 'f_name', 'type'], dtype='object')

In [495]:
final_df=pd.merge(food,menu,on=['f_id'],how='inner')
average_ppd=final_df.groupby('f_name')['price'].mean().reset_index()
average_ppd
average_ppd=average_ppd.rename(columns={'price':'average_price_per_dish'})
average_ppd

f_name  average_price_per_dish
0    Chicken Popcorn              300.000000
1      Chicken Wings              230.000000
2    Choco Lava cake               98.333333
3        Masala Dosa              180.000000
4      Non-veg Pizza              450.000000
5          Rava Idli              120.000000
6          Rice Meal              213.333333
7          Roti meal              140.000000
8   Schezwan Noodles              220.000000
9     Veg Manchurian              180.000000
10         Veg Pizza              400.000000

In [496]:
final_df=final_df.merge(average_ppd,on=['f_name'],how='left')
final_df

f_id            f_name     type  menu_id  r_id  price  \
0      1     Non-veg Pizza  Non-veg        1     1    450   
1      2         Veg Pizza      Veg        2     1    400   
2      3   Choco Lava cake      Veg        3     1    100   
3      3   Choco Lava cake      Veg        4     2    115   
4      3   Choco Lava cake      Veg        7     3     80   
5      4     Chicken Wings  Non-veg        5     2    230   
6      5   Chicken Popcorn  Non-veg        6     2    300   
7      6         Rice Meal      Veg        8     3    160   
8      6         Rice Meal      Veg       10     4    230   
9      6         Rice Meal      Veg       13     5    250   
10     7         Roti meal      Veg        9     3    140   
11     8       Masala Dosa      Veg       11     4    180   
12     9         Rava Idli      Veg       12     4    120   
13    10  Schezwan Noodles      Veg       14     5    220   
14    11    Veg Manchurian      Veg       15     5    180   

    average_price_per_dish  
0               450.000000  
1               400.000000  
2                98.333333  
3                98.333333  
4                98.333333  
5               230.000000  
6               300.000000  
7               213.333333  
8               213.333333  
9               213.333333  
10              140.000000  
11              180.000000  
12              120.000000  
13              220.000000  
14              180.000000

 # Find top restautant in terms of number of orders for a given month

In [497]:
order.columns

Index(['order_id', 'user_id', 'r_id', 'amount', 'date', 'partner_id',
       'delivery_time', 'delivery_rating', 'restaurant_rating'],
      dtype='object')

In [498]:
resturant.columns

Index(['r_id', 'r_name', 'cuisine'], dtype='object')

In [499]:
order['date']= pd.to_datetime(order['date'])
order['date']

0    2022-05-10
1    2022-05-26
2    2022-06-15
3    2022-06-29
4    2022-07-10
5    2022-06-10
6    2022-06-23
7    2022-07-07
8    2022-07-17
9    2022-07-31
10   2022-05-10
11   2022-05-20
12   2022-05-30
13   2022-06-11
14   2022-06-22
15   2022-05-15
16   2022-05-30
17   2022-06-15
18   2022-06-30
19   2022-07-15
20   2022-07-01
21   2022-07-08
22   2022-07-15
23   2022-07-21
24   2022-07-28
Name: date, dtype: datetime64[ns]

In [500]:

for i in range(5,8): 
    target_month = (f'2022-0{i}')
    monthly_orders = order[order['date'].dt.to_period('M') == target_month]
    
    top_resturant = (monthly_orders.groupby('r_id')
                      .size()
                      .reset_index(name='order_count')
                      .sort_values(by='order_count', ascending=False)
                      .head(1)
                      .merge(resturant, on='r_id'))
    print(f'\033[1m Top resturant of given month "{target_month}" are: \n \033[0m {top_resturant}')


 Top resturant of given month "2022-05" are: 
     r_id  order_count      r_name       cuisine
0     4            3  Dosa Plaza  South Indian
 Top resturant of given month "2022-06" are: 
     r_id  order_count r_name   cuisine
0     2            3    kfc  American
 Top resturant of given month "2022-07" are: 
     r_id  order_count r_name   cuisine
0     2            3    kfc  American


# restaurants with monthly sales > 4000

In [515]:
order_details.columns

Index(['id', 'order_id', 'f_id'], dtype='object')

In [502]:
order.columns

Index(['order_id', 'user_id', 'r_id', 'amount', 'date', 'partner_id',
       'delivery_time', 'delivery_rating', 'restaurant_rating'],
      dtype='object')

In [503]:
food.columns

Index(['f_id', 'f_name', 'type'], dtype='object')

In [504]:
resturant.columns

Index(['r_id', 'r_name', 'cuisine'], dtype='object')

In [505]:
resturant

r_id      r_name       cuisine
0     1     dominos       Italian
1     2         kfc      American
2     3        box8  North Indian
3     4  Dosa Plaza  South Indian
4     5  China Town       Chinese

In [506]:
fin_order = order_details.merge(food, on='f_id')
fin_order = fin_order.merge(order[['order_id', 'date', 'r_id']], on='order_id')


In [507]:
fin_order['date'] = pd.to_datetime(fin_order['date'])
fin_order['month'] = fin_order['date'].dt.to_period('M')
fin_order



id  order_id  f_id            f_name     type       date  r_id    month
0    1      1001     1     Non-veg Pizza  Non-veg 2022-05-10     1  2022-05
1    2      1001     3   Choco Lava cake      Veg 2022-05-10     1  2022-05
2    3      1002     4     Chicken Wings  Non-veg 2022-05-26     2  2022-05
3    4      1002     3   Choco Lava cake      Veg 2022-05-26     2  2022-05
4    5      1003     6         Rice Meal      Veg 2022-06-15     3  2022-06
5    6      1003     3   Choco Lava cake      Veg 2022-06-15     3  2022-06
6    7      1004     6         Rice Meal      Veg 2022-06-29     3  2022-06
7    8      1004     3   Choco Lava cake      Veg 2022-06-29     3  2022-06
8    9      1005     7         Roti meal      Veg 2022-07-10     3  2022-07
9   10      1005     3   Choco Lava cake      Veg 2022-07-10     3  2022-07
10  11      1006     1     Non-veg Pizza  Non-veg 2022-06-10     1  2022-06
11  12      1006     2         Veg Pizza      Veg 2022-06-10     1  2022-06
12  13      1006     3   Choco Lava cake      Veg 2022-06-10     1  2022-06
13  14      1007     4     Chicken Wings  Non-veg 2022-06-23     2  2022-06
14  15      1007     3   Choco Lava cake      Veg 2022-06-23     2  2022-06
15  16      1008     6         Rice Meal      Veg 2022-07-07     3  2022-07
16  17      1008     3   Choco Lava cake      Veg 2022-07-07     3  2022-07
17  18      1009     8       Masala Dosa      Veg 2022-07-17     4  2022-07
18  19      1009     9         Rava Idli      Veg 2022-07-17     4  2022-07
19  20      1010    10  Schezwan Noodles      Veg 2022-07-31     5  2022-07
20  21      1010    11    Veg Manchurian      Veg 2022-07-31     5  2022-07
21  22      1010     6         Rice Meal      Veg 2022-07-31     5  2022-07
22  23      1011     1     Non-veg Pizza  Non-veg 2022-05-10     1  2022-05
23  24      1012     8       Masala Dosa      Veg 2022-05-20     4  2022-05
24  25      1013     4     Chicken Wings  Non-veg 2022-05-30     2  2022-05
25  26      1014     4     Chicken Wings  Non-veg 2022-06-11     2  2022-06
26  27      1015     4     Chicken Wings  Non-veg 2022-06-22     2  2022-06
27  28      1016     8       Masala Dosa      Veg 2022-05-15     4  2022-05
28  29      1016     9         Rava Idli      Veg 2022-05-15     4  2022-05
29  30      1017     8       Masala Dosa      Veg 2022-05-30     4  2022-05
30  31      1017     9         Rava Idli      Veg 2022-05-30     4  2022-05
31  32      1018    10  Schezwan Noodles      Veg 2022-06-15     4  2022-06
32  33      1018    11    Veg Manchurian      Veg 2022-06-15     4  2022-06
33  34      1019    10  Schezwan Noodles      Veg 2022-06-30     5  2022-06
34  35      1019    11    Veg Manchurian      Veg 2022-06-30     5  2022-06
35  36      1020    10  Schezwan Noodles      Veg 2022-07-15     5  2022-07
36  37      1020    11    Veg Manchurian      Veg 2022-07-15     5  2022-07
37  38      1021     1     Non-veg Pizza  Non-veg 2022-07-01     1  2022-07
38  39      1021     3   Choco Lava cake      Veg 2022-07-01     1  2022-07
39  40      1022     1     Non-veg Pizza  Non-veg 2022-07-08     1  2022-07
40  41      1022     3   Choco Lava cake      Veg 2022-07-08     1  2022-07
41  42      1023     3   Choco Lava cake      Veg 2022-07-15     2  2022-07
42  43      1023     4     Chicken Wings  Non-veg 2022-07-15     2  2022-07
43  44      1023     5   Chicken Popcorn  Non-veg 2022-07-15     2  2022-07
44  45      1024     3   Choco Lava cake      Veg 2022-07-21     2  2022-07
45  46      1024     4     Chicken Wings  Non-veg 2022-07-21     2  2022-07
46  47      1024     5   Chicken Popcorn  Non-veg 2022-07-21     2  2022-07
47  48      1025     3   Choco Lava cake      Veg 2022-07-28     2  2022-07
48  49      1025     4     Chicken Wings  Non-veg 2022-07-28     2  2022-07
49  50      1025     5   Chicken Popcorn  Non-veg 2022-07-28     2  2022-07

In [508]:
fin_order['total_price'] = order['amount']



In [509]:


monthly_sales = (fin_order.groupby(['r_id', 'month'])['total_price']
                 .sum()
                 .reset_index())
monthly_sales


r_id    month  total_price
0      1  2022-05       1610.0
1      1  2022-06        860.0
2      1  2022-07          0.0
3      2  2022-05       1125.0
4      2  2022-06        460.0
5      2  2022-07          0.0
6      3  2022-06       1940.0
7      3  2022-07       1550.0
8      4  2022-05        645.0
9      4  2022-06          0.0
10     4  2022-07        800.0
11     5  2022-06          0.0
12     5  2022-07       1500.0

In [510]:
high_sales = monthly_sales[monthly_sales['total_price'] > 4000]
high_sales = high_sales.merge(resturant, on='r_id')
high_sales
print(f'\033[1m here is no Restaurants with monthly sales > 4000 \033[0m \n {high_sales}')

 here is no Restaurants with monthly sales > 4000  
 Empty DataFrame
Columns: [r_id, month, total_price, r_name, cuisine]
Index: []


# Show all orders with order details for a particular customer in a particular date range

In [511]:
users


user_id      name               email password
0        1    Nitish    nitish@gmail.com    p252h
1        2  Khushboo  khushboo@gmail.com    hxn9b
2        3   Vartika   vartika@gmail.com    9hu7j
3        4     Ankit     ankit@gmail.com    lkko3
4        5      Neha      neha@gmail.com    3i7qm
5        6   Anupama   anupama@gmail.com   46rdw2
6        7   Rishabh   rishabh@gmail.com   4sw123

In [512]:
food.columns

Index(['f_id', 'f_name', 'type'], dtype='object')

In [516]:
start_date = '2022-05-01'
end_date = '2022-05-31'
target_user_id = 1

orders_filtered = order[(order['user_id'] == target_user_id) &
                         (order['date'].between(start_date, end_date))]

order_details_user = (order_details.merge(orders_filtered[['order_id', 'date']], on='order_id')
                      .merge(food[['f_id', 'f_name', ]], on='f_id'))

order_details_user['total_price'] = order['amount']
order_details_user

id  order_id  f_id       date           f_name  total_price
0   1      1001     1 2022-05-10    Non-veg Pizza          550
1   2      1001     3 2022-05-10  Choco Lava cake          415
2   3      1002     4 2022-05-26    Chicken Wings          240
3   4      1002     3 2022-05-26  Choco Lava cake          240

# Find restaurants with max repeated customers

In [519]:
repeats = order.groupby(['r_id', 'user_id']).size().reset_index(name='order_count')
repeats = repeats[repeats['order_count'] > 1]
repeats



r_id  user_id  order_count
3      1        5            2
6      2        3            3
7      2        5            3
8      3        1            3
12     4        4            3
14     5        4            2

In [520]:
repeated_counts = repeats.groupby('r_id')['user_id'].nunique().reset_index(name='repeat_customers')
repeated_counts

r_id  repeat_customers
0     1                 1
1     2                 2
2     3                 1
3     4                 1
4     5                 1

In [521]:
top_repeat_restaurant = repeated_counts.sort_values(by='repeat_customers', ascending=False).head(1)
top_repeat_restaurant = top_repeat_restaurant.merge(resturant, on='r_id')
top_repeat_restaurant

r_id  repeat_customers r_name   cuisine
0     2                 2    kfc  American

In [525]:
print("\033[1m KFC is most repeated restaurant.\033[1m")

 KFC is most repeated restaurant.


# Month over month revenue growth of swiggy

In [526]:
order_details.columns


Index(['id', 'order_id', 'f_id'], dtype='object')

In [529]:
fin_order['revenue'] =order['amount']
fin_order['month'] = order['date'].dt.to_period('M')

monthly_revenue = (fin_order.groupby('month')['revenue']
                   .sum()
                   .reset_index())
monthly_revenue['mom_growth_%'] = monthly_revenue['revenue'].pct_change().fillna(0) * 100
monthly_revenue['mom_growth_%']

0     0.000000
1    32.783505
2    50.465839
Name: mom_growth_%, dtype: float64

# Customer -> favorite food

In [530]:
order_details.columns


Index(['id', 'order_id', 'f_id'], dtype='object')

In [532]:
food.columns

Index(['f_id', 'f_name', 'type'], dtype='object')

In [537]:
user_food_counts = (order[['order_id', 'user_id']]
                    .merge(order_details[['order_id', 'f_id']], on='order_id')
                    .merge(food[['f_id', 'f_name']], on='f_id'))

user_food_counts.columns

Index(['order_id', 'user_id', 'f_id', 'f_name'], dtype='object')

In [541]:
fav_food = (user_food_counts.groupby(['user_id', 'f_name'])
            .size()
            .reset_index(name='order_count'))

fav_food.columns

Index(['user_id', 'f_name', 'order_count'], dtype='object')

In [542]:
fav_food['rank'] = fav_food.groupby('user_id')['order_count'].rank(method='first', ascending=False)
user_fav_food = fav_food[fav_food['rank'] == 1]

In [543]:
user_fav_food

user_id            f_name  order_count  rank
1         1   Choco Lava cake            5   1.0
6         2   Choco Lava cake            3   1.0
14        3     Chicken Wings            3   1.0
19        4  Schezwan Noodles            3   1.0
23        5   Choco Lava cake            5   1.0